# HW01 PLA
> 1041621 蘇建文

## The roadmap of code
![](https://raw.githubusercontent.com/johnny12150/DSS_P2P/f3eabbf965fea515e87094703e00d9c34b3bac6b/notebook/img/Slide10.jpg)

### 01. Dataset
> 資料來源: https://www.kaggle.com/c/titanic/data

檢視資料

In [1]:
import numpy as np
import pandas as pd

DF = pd.read_csv('train.csv')
DF.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


### 02. 資料前處理

#### 選取feature & label

In [2]:
train_feature = DF.drop(['Name', 'Sex', 'Ticket', 'PassengerId','Survived','Cabin'], axis=1)
train_labels = DF['Survived'].copy()

#### 處理空值、異常值 
檢查空值

In [3]:
train_feature.isnull().sum()

Pclass        0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

將年紀的空值用平均值補

In [3]:
median_age = train_feature['Age'].dropna().median()
# 取出Age是Nan的row後，將該row的Age以平均數取代之
train_feature.loc[ (train_feature['Age'].isnull()), 'Age'] = median_age

用眾數補embarked

In [4]:
train_feature['Embarked'] = train_feature['Embarked'].fillna(train_feature['Embarked'].mode()[0])

### 03. One Hot Encoding
將Embarked mapping成數值

In [5]:
train_feature['Embarked'] = train_feature['Embarked'].map({'C': 0, 'Q': 1, 'S':2}).astype(int)

### 05. 演算法

### 1. PLA演算法

參數設定

In [18]:
weights = np.zeros(len(train_feature.columns))
print(weights)

# weights.fill(0)

learningRate = 0.05

Iterations = 10
# 紀錄error變化
errors = []

[0. 0. 0. 0. 0. 0.]


激勵函數

In [ ]:
def sign(z):
    if z > 0:
        return 1
    else:
        return -1

簡單版

In [17]:
def pla_simple(data, labels, iterations):
    for numIter in range(Iterations):
        print("iteration {}".format(numIter))
        err = 0  # reset the error counter
        # For each handwritten digit in training set,
        for ix in range(len(data.index)):
            # select a row as input vector 
            inputVector = data.iloc[ix]
            # uses global variable in the function
            global weights
            # take the dot product of input and weight
            dotproduct = np.dot(inputVector, weights)
            # 經過激勵函數
            estimatedLabel = np.sign(dotproduct)
            
            # 如果sign回傳是0, 看作小於0
            if estimatedLabel == 0:
                estimatedLabel = -1
            actualLabel = labels[ix];
            # 比較預測的label與答案
            if actualLabel != estimatedLabel:
#                 weights = weights + actualLabel*learningRate*inputVector
                # actualLabel - estimatedLabel 計算 loss
                weights = weights + (actualLabel - estimatedLabel)*inputVector
                err+=1
                
        global errors
        errors.append( (err*1.0) / len(data) )  
        print(weights)

In [19]:
pla_simple(train_feature, train_labels, Iterations)
errors

iteration 0
Pclass      -10.0000
Age         -35.5000
SibSp       -98.0000
Parch       -21.0000
Fare        107.5869
Embarked    -17.0000
dtype: float64
iteration 1
Pclass      -30.0000
Age         -41.5000
SibSp      -132.0000
Parch       -10.0000
Fare         95.2662
Embarked    -64.0000
dtype: float64
iteration 2
Pclass      -38.0000
Age          -5.5000
SibSp      -170.0000
Parch       -33.0000
Fare         -8.6631
Embarked    -91.0000
dtype: float64
iteration 3
Pclass      -42.0000
Age         -25.5000
SibSp      -244.0000
Parch       -78.0000
Fare         81.5914
Embarked   -108.0000
dtype: float64
iteration 4
Pclass       -4.0000
Age         -11.5000
SibSp      -310.0000
Parch      -111.0000
Fare         15.6035
Embarked   -129.0000
dtype: float64
iteration 5
Pclass       26.000
Age         -41.500
SibSp      -270.000
Parch       -88.000
Fare         95.016
Embarked   -130.000
dtype: float64
iteration 6
Pclass       42.0000
Age         -28.0000
SibSp      -278.0000
Parch       -

[0.6992143658810326,
 0.696969696969697,
 0.7048260381593715,
 0.7104377104377104,
 0.7059483726150393,
 0.7093153759820426,
 0.7014590347923682,
 0.7104377104377104,
 0.712682379349046,
 0.7115600448933782]

訓練後的權重

In [22]:
weights

Pclass      0.010000
Age         0.380000
SibSp       0.010000
Parch       0.000000
Fare        0.712833
Embarked    0.000000
dtype: float64

到特定iteration停下

In [ ]:
def pla(data):
    w = np.array([0.,0.,0.])
    error = 1
    iterator = 0
    while error != 0:
        error = 0
        for i in range(len(data)):
            x, y = np.concatenate((np.array([1.]), np.array(data.iloc[i])[:2])), np.array(data.iloc[i])[2]
            if sign(np.dot(w,x)) != y:
                print("iterator: "+str(iterator))
                iterator += 1
                error += 1
                

                # 前一個 Decision boundary 的法向量
                if w[1] != 0:
                    x_last_decision_boundary = np.linspace(0,w[1])
                    y_last_decision_boundary = (w[2]/w[1])*x_last_decision_boundary
                    
                w += y*x            
                print("x: " + str(x))            
                print("w: " + str(w))
                # x向量 
                x_vector = np.linspace(0,x[1])
                y_vector = (x[2]/x[1])*x_vector
                
                # Decision boundary 的方向向量
                x_decision_boundary = np.linspace(-0.5,7)
                y_decision_boundary = (-w[1]/w[2])*x_decision_boundary - (w[0]/w[2])
                
                # Decision boundary 的法向量
                x_decision_boundary_normal_vector = np.linspace(0,w[1])
                y_decision_boundary_normal_vector = (w[2]/w[1])*x_decision_boundary_normal_vector

In [ ]:
pla(train_feature)

替代方案

In [ ]:
def naive_pla(datas):
    # 初始化權重,全給0
    w = datas[0][0]
    iteration = 0
    while True:
        iteration += 1
        false_data = 0

        for data in datas:
            # input與權重做內積
            t = dot(w, data[0])
            if sign(data[1]) != sign(t):
                error = data[1]
                false_data += 1
                w += error * data[0]
        print 'iter%d (%d / %d)' % (iteration, false_data, len(datas))
        if not false_data:
            break
    return w

In [ ]:
naive_pla()

### 2. PLA Pocket演算法

In [ ]:
def pocket_pla(datas, limit):
    
    
    def _calc_false(vec):
        res = 0
        for data in datas:
            t = np.dot(vec, data[0])
            if np.sign(data[1]) != np.sign(t):
                res += 1
        return res
    
    
    w = np.random.rand(5)
    least_false = _calc_false(w)
    res = w

    for i in xrange(limit):
        data = random.choice(datas)
        t = np.dot(w, data[0])
        if np.sign(data[1]) != np.sign(t):
            t = w + data[1] * data[0]
            t_false = _calc_false(t)

            w = t

            if t_false <= least_false:
                least_false = t_false
                res = t
    return res, least_false

In [ ]:
pocket_pla()

## 預測

In [14]:
def pla_predict(data):
    predict = []
    for ix in range(len(data.index)):
        inputVector = data.iloc[ix]
        global weights
        dotproduct = np.dot(inputVector, weights)  # take the dot product of input and weight
        estimatedLabel = np.sign(dotproduct)
        if estimatedLabel == -1:
            estimatedLabel = 0
        else:
            estimatedLabel = 1
        predict.append(estimatedLabel) 
        
    return predict

In [20]:
test = pd.read_csv('test.csv')

test_feature = test.drop(['Name', 'Sex', 'Ticket', 'PassengerId','Cabin'], axis=1)
median_age = test_feature['Age'].dropna().median()
test_feature.loc[ (test_feature['Age'].isnull()), 'Age'] = median_age
test_feature['Embarked'] = test_feature['Embarked'].fillna(test_feature['Embarked'].mode()[0])
test_feature['Embarked'] = test_feature['Embarked'].map({'C': 0, 'Q': 1, 'S':2}).astype(int)

y_pred = pla_predict(test_feature)

### 計算模型準確度

In [21]:
from sklearn.metrics import accuracy_score
answer = pd.read_csv('submission.csv')

accuracy_score(answer['Survived'], y_pred)

0.6842105263157895